# Asynchronous Scraping

In [2]:
#!pip install aiohttp


In [3]:
!pip install asyncio

  Obtaining dependency information for asyncio from https://files.pythonhosted.org/packages/22/74/07679c5b9f98a7cb0fc147b1ef1cc1853bc07a4eb9cb5731e24732c5f773/asyncio-3.4.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/101.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/101.8 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/101.8 kB ? eta -:--:--
   --------------- ----------------------- 41.0/101.8 kB 326.8 kB/s eta 0:00:01
   ----------------------------------- --- 92.2/101.8 kB 655.4 kB/s eta 0:00:01
   -------------------------------------- 101.8/101.8 kB 586.7 kB/s eta 0:00:00


In [4]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import csv
import re

In [5]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

In [6]:
async def scrap_and_save_links(text):
  soup = BeautifulSoup(text, 'html.parser')
  file = open('csv_file', 'a', newline='')
  writer= csv.writer(file, delimiter=',')
  for link in soup.findAll('a', attrs={'href': re.compile('^http')}):
    link = link.get('href')
    writer.writerow([link])
  file.close()

In [7]:
async def fetch(session, url):
   try:
      async with session.get(url) as response:
         text= await response.text()
         task = asyncio.create_task(scrap_and_save_links(text))
         await task
   except Exception as e:
      print(str(e))

In [8]:
async def scrap(urls):
  tasks = []
  async with aiohttp.ClientSession() as session:
    for url in urls:
      tasks.append(fetch(session,url))
    await asyncio.gather(*tasks)

In [9]:
urls = ['https://analytics.usa.gov/', 'https://www.python.org/', 'https://www.linkedin.com/']
asyncio.run(scrap(urls=urls))